In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 28 16:20:48 2017

@author: moonsooyoung
"""

import os
import numpy as np

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import matplotlib.pylab as plt
import PIL.Image as pilimg


/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#학습시킬 MNIST 숫자데이터를 부름.
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
#DeepNeuralNetwork에 쓸 각 layer에 사용할 노드의 갯수를 사용자가 원하는 만큼 정해줌.
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

#n_classes = 데이터의 클래스 수. MNIST에서는 0,1,2,3,4,5,6,7,8,9 총 10개.
n_classes = 10

#batch_size를 원하는 만큼 정해줌.
batch_size = 100

In [4]:
#input과 output의 placeholder을 만들어줌. [None, 784]는 input data의 사이즈.
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

#신경망 구조에 들어갈 변수를 정의해줌. 훈련 전에는 랜덤한 값을 임의로 넣어 초기값을 잡아줌.
#y=XW+b 임을 고려하여 벡터크기를 지정해줌.
w1 = tf.Variable(tf.random_normal([784, n_nodes_hl1]))
b1 = tf.Variable(tf.random_normal([n_nodes_hl1]))

w2 = tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2]))
b2 = tf.Variable(tf.random_normal([n_nodes_hl2]))

w3 = tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3]))
b3 = tf.Variable(tf.random_normal([n_nodes_hl3]))

w4 = tf.Variable(tf.random_normal([n_nodes_hl3, n_classes]))
b4 = tf.Variable(tf.random_normal([n_classes]))

In [5]:
#신경망 구조
l1 = tf.add(tf.matmul(x,w1), b1)
l1 = tf.nn.relu(l1)

l2 = tf.add(tf.matmul(l1,w2), b2)
l2 = tf.nn.relu(l2)

l3 = tf.add(tf.matmul(l2,w3), b3)
l3 = tf.nn.relu(l3)

output = tf.matmul(l3,w4) + b4

In [6]:
#변수들을 저장해줄 파일경로를 지정해줌.
save_path = 'machine/'
model_name = 'DNN_md'
if not os.path.exists(save_path):
    os.makedirs(save_path)

save_path_full = os.path.join(save_path, model_name)

In [7]:
#학습 단계 알고리즘
def train_neural_network(x):
    prediction = output
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
    learning_rate=0.001
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    hm_epochs = 5
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                sess.run(l1, feed_dict={x: epoch_x})
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
        #아까 지정한 파일경로로 변수들을 저장해줌. 지정할 변수를 리스트 형태로 써줌.
        saver = tf.train.Saver([w1, w2, w3, w4, b1, b2, b3, b4])
        saver.save(sess,save_path_full)    #자신이 지정해준 파일에 .meta형식의 파일이 생김.
        
#학습 시작
train_neural_network(x)

Epoch 0 completed out of 5 loss: 1622345.8783
Epoch 1 completed out of 5 loss: 385332.328979
Epoch 2 completed out of 5 loss: 214236.015845
Epoch 3 completed out of 5 loss: 127034.356157
Epoch 4 completed out of 5 loss: 78192.3934943
Accuracy: 0.9398


In [8]:
#테스트 단계
#테스트할 파일을 불러옴.
k = pilimg.open('/Volumes/SM/seven.png' )
plt.imshow(k)
imgarray=np.array(k)    #컬러채널이 1개가 되도록 이미 전처리를 한 상태라서 그냥 (28,28)로 나옴.
kkk = imgarray/255    #k1 벡터 안의 숫자들을 0과 1 사이로 normalize시키기 벡터 내의 가장 큰 값으로 k1을 나눠줌.
sydata=kkk.reshape(1,784)    #훈련할 때와 같은 input data 형태로 맞춰줌. (28,28)->(1,784).
x_data = tf.cast(sydata, 'float')

In [9]:
#변수 불러오기
sess=tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver([w1, w2, w3, w4, b1, b2, b3, b4])    #전에 저장해둔 변수를 불러옴.
new_saver = tf.train.import_meta_graph('/Volumes/SM/DNN/machine/DNN_md.meta')    
#학습 단계에서 자동으로 생긴 meta file에 변수들의 값이 저장되어 있음. saver.restore()로 그 값들을 불러옴.
saver.restore(sess,save_path_full)

INFO:tensorflow:Restoring parameters from machine/DNN_md


In [10]:
#처음에 썼던 신경망 알고리즘을 그대로 써주고, input data넣는 x자리에 테스트 할 데이터(x_data)를 바꿔 써줌.
l1 = tf.add(tf.matmul(x_data,w1), b1)
l1 = tf.nn.relu(l1)

l2 = tf.add(tf.matmul(l1,w2), b2)
l2 = tf.nn.relu(l2)

l3 = tf.add(tf.matmul(l2,w3), b3)
l3 = tf.nn.relu(l3)

output = tf.matmul(l3,w4) + b4

In [11]:
#테스트 시작
print('============================================TEST 결과============================================')
print(sess.run (output))
print(sess.run(tf.argmax(output, 1)))

============================================TEST 결과============================================
[[ -1691.77246094  -9482.28320312   7775.19384766  13715.46289062
   -4566.18164062  -9743.04394531 -28360.29492188  23831.29882812
   10846.51367188   4017.27563477]]
[7]
